In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout

import sklearn as sk
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier 
from sklearn import svm

Using Theano backend.


# Data preprocessing

In [2]:
#load dataset
dataframe = pd.DataFrame.from_csv('base_train.csv',sep=';',index_col=None)

Check that all variables are binary.

In [3]:
# check that sum of 1 and 0 values equals to the dimension of the data
dataset = dataframe.values
sum((dataset == 0)*1)+sum((dataset == 1)*1)

array([    0,     0, 42602, 42602, 42602, 42602, 42602, 42602, 42602,
       42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602,
       42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602,
       42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602,
       42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602, 42602,
       42602, 42602, 42602, 42602, 42602, 42602, 42602, 31083, 31083])

First two columns include date and ID correspondingly. To incorporate date into analysis we replace it by the month number. Thus we'll have a new variable with 12 values which reflects dependency of the output from the month.

In [4]:
# convert dates to datetime format
dataframe['report_dt'] = pd.to_datetime(dataframe['report_dt'])
# extract month numbers
month = numpy.zeros(len(dataframe))
for i in range(len(dataframe)):
    month[i] = dataframe['report_dt'][i].month
#replace date with month number
dataframe['report_dt'] = month

Find missing data.

In [5]:
# check the number of missing inputs and outputs
dataframe.isnull().sum()

report_dt        0
ID               0
1.1.             0
1.2.             0
1.3.             0
1.4.             0
1.5.             0
1.6.             0
1.7.             0
1.8.             0
1.9.             0
1.10.            0
1.11.            0
1.12.            0
1.13.            0
1.14.            0
1.15.            0
1.16.            0
1.17.            0
1.18.            0
1.19.            0
2.1.             0
2.2.             0
2.3.             0
2.4.             0
2.5.             0
2.6.             0
2.7.             0
2.8.             0
2.9.             0
2.10.            0
2.11.            0
2.12.            0
2.13.            0
2.14.            0
2.15.            0
2.16.            0
3.1.             0
3.2.             0
3.3.             0
3.4.             0
3.5.             0
3.6.             0
3.7.             0
3.8.             0
3.9.             0
3.10.            0
3.11.            0
3.12.            0
3.13.            0
3.14.            0
3.15.            0
X3          

There are no missing inputs, but about 25% of outputs are missing. We need to exclude those records from the dataset, as they aren't valid both for trainig and testing.

In [6]:
# delete data with missing values
dataframe = dataframe.dropna()

Next we test how the dataset is balanced, i.e. what is the proportion of 0 and 1 values of output variables. 

In [7]:
# number of 1 values in X3 variable
sum((dataframe['X3'] ==1)*1)

966

In [8]:
# number of 1 values in Y3 variable
sum((dataframe['Y3'] ==1)*1)

669

We see that value 1 is obtained by less then 1/30 of the dataset. Which means that we'll need to apply different weights to 0 and 1 during trainig to balance classes.

As there's no a priori information about dependencies in the date we delete column ID. Also as we don't know the nature of the output variables (if ther's any connection between them) we use separate models to predict X3 and Y3. Next we predict X3 variable (to predict Y3 change "del dataframe['Y3']" to "del dataframe['X3']" in the code box below).

In [9]:
del dataframe['report_dt']
del dataframe['ID']
del dataframe['Y3']

Next we select features that have the highest scores using chi2 function for scoring. However we should note that this is not required if we use a neural network (usually it's assumed that this step is performed by the network itself).

In [10]:
# calculate scores of featuress
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
values = SelectKBest(chi2, k='all').fit(dataframe.drop('X3',axis=1).values,dataframe['X3'].values)
scores = -numpy.log10(values.pvalues_)

# uncomment code below to plot results
#%matplotlib inline
#plt.bar(range(len(dataframe.columns)-1), scores)
#plt.xticks(range(len(dataframe.columns)-1), dataframe.columns, rotation='vertical')
#plt.show()

Now we leave 30 features with highest scores and drop others.

In [11]:
# select K best features and transform dataset
dataset_reduced = SelectKBest(chi2, k=30).fit_transform(dataframe.drop('X3',axis=1).values,dataframe['X3'].values)
dataset_reduced = numpy.append(dataset_reduced,numpy.reshape(dataframe['X3'].values,(len(dataframe['X3'].values),1)),axis=1)

In the last step we split data into training and test sets in proportion 3/1.

In [12]:
# split into trainig (75%) and test (25%) sets
train, test = sk.cross_validation.train_test_split(dataset_reduced, test_size = 0.25)
# split into input (X) and (Y) variables
Xtrain = train[:,0:train.shape[1]-1]
Ytrain = train[:,train.shape[1]-1]
Xtest = test[:,0:test.shape[1]-1]
Ytest = test[:,test.shape[1]-1]

# Neural network

We build a fully-connected network with three layers. The network weights are initialized to a small random number drawn from a uniform distribution. As an activation function on the first two layers we use the rectifier function 'relu'. In the last layer we use sigmoid to insure the output is between 0 and 1. First layer has 35 neurons, 2nd layer 24 neurons, the output layer has 1 neuron to predict the output. Between hiden layers we also perform dropout operation for better distribution of weights between neurons.

In [13]:
# create model
model = Sequential()
model.add(Dense(35, input_dim=train.shape[1]-1, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(24, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, init='uniform', activation='sigmoid'))

To find the best set of weights we must specify: the loss function, optimizer and metrics that we want to calculate during training. For binary classification problem we pick "binary_crossentropy" loss. The optimization method is gradient descent "adadelta". We calculate classification accuracy as a metric.

In [14]:
# set model
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

Here we train the model. We pick 300 iterations for the optimization algorithm and a batch size of 32. Each epoch separates 20% of data for validation.

In [15]:
# train the model
model.fit(Xtrain,Ytrain, validation_split=0.2, nb_epoch=300, batch_size=32, class_weight="balanced", verbose=0)

Finally, after the model is ready, we generate predicitons on the test set.

In [16]:
# make prediction on test set
Ypred= model.predict_classes(Xtest)

7040/7771 [==========================>...] - ETA: 0s

# Decision tree

Here we build the decision tree model using sklearn framework.

In [17]:
# create and fit model
modeltree = tree.DecisionTreeClassifier()
modeltree.fit(Xtrain, Ytrain)

# make prediction on test set
Ypredtree = modeltree.predict(Xtest)

# Random forest 

Random forest is a batch of trees, we'll check it's perpormance too.

In [18]:
# Create the random forest
modelforest = RandomForestClassifier(n_estimators = 100)
modelforest.fit(Xtrain,Ytrain)

# make prediction on test set
Ypredforest = modelforest.predict(Xtest)

# SVM

The next model is the SVM classifier.

In [19]:
# create and fit model
modelsvm = svm.SVC()
modelsvm.fit(Xtrain, Ytrain)

# make prediction on test set
Ypredsvm = modelsvm.predict(Xtest)

# Log regression

In [20]:
modelreg = sk.linear_model.LogisticRegression()
modelreg.fit(Xtrain,Ytrain)

Ypredreg = modelreg.predict(Xtest)

# Evaluate the models

Here we define a function that evaluates performance of the model through different metrics. Short description for each metric is given in the code below.

In [21]:
def evaluation(Xtest,Ytest,Ypred):
    
    # accuracy on test set
    score = sk.metrics.accuracy_score(Ytest,Ypred)
    print 'accuracy:', round(score*100,2), '%'
    
    # precision on test set
    # The precision is the ratio tp / (tp + fp) where tp is the
    # number of true positives and fp the number of false positives.
    # The best value is 1 and the worst value is 0.
    score = sk.metrics.precision_score(Ytest,Ypred)
    print 'precision:', round(score*100,2), '%' 
    
    # recall on test set
    # The recall is the ratio tp / (tp + fn) where tp is the number of
    # true positives and fn the number of false negatives.
    # The best value is 1 and the worst value is 0.
    score = sk.metrics.recall_score(Ytest,Ypred)
    print 'recall:', round(score*100,2), '%'
    
    # f1 score on test set
    # The F1 score can be interpreted as a weighted average of the 
    # precision and recall, where an F1 score reaches its best value at
    # 1 and worst score at 0.
    score = sk.metrics.f1_score(Ytest,Ypred)
    print 'f1:', score
    
    # Area Under the Curve from prediction scores
    # The best performance is 1
    score = sk.metrics.roc_auc_score(Ytest,Ypred)
    print 'AUC:', score  
    
    # Confusion matrix
    # By definition a confusion matrix C is such that C_{i, j} 
    # is equal to the number of observations known to be in group i 
    # but predicted to be in group j.
    print 'Confusion matrix:'
    print sk.metrics.confusion_matrix(Ytest,Ypred)
    
    # Matthews correlation coefficient (phi coefficient)
    # Is used in machine learning 
    # as a measure of the quality of binary (two-class) classifications.
    # It takes into account true and false positives and negatives and 
    # is generally regarded as a balanced measure which can be used even
    # if the classes are of very different sizes. 
    #+1 perfect prediction; 0 random prediction; -1 inverse prediction
    score = sk.metrics.matthews_corrcoef(Ytest,Ypred)
    print 'MCC:', score

Now lets evaluate each model performance. First goes the neural network:

In [22]:
evaluation(Xtest,Ytest,Ypred)

accuracy: 98.53 %
precision: 82.78 %
recall: 64.22 %
f1: 0.723300970874
AUC: 0.819064714062
Confusion matrix:
[[7508   31]
 [  83  149]]
MCC: 0.721977875198


Now the decision tree:

In [23]:
evaluation(Xtest,Ytest,Ypredtree)

accuracy: 98.52 %
precision: 80.95 %
recall: 65.95 %
f1: 0.726840855107
AUC: 0.82735379475
Confusion matrix:
[[7503   36]
 [  79  153]]
MCC: 0.723311628753


Random forest:

In [24]:
evaluation(Xtest,Ytest,Ypredforest)

accuracy: 98.57 %
precision: 82.01 %
recall: 66.81 %
f1: 0.736342042755
AUC: 0.831796783164
Confusion matrix:
[[7505   34]
 [  77  155]]
MCC: 0.73312872951


The SVM:

In [25]:
evaluation(Xtest,Ytest,Ypredsvm)

accuracy: 98.12 %
precision: 72.4 %
recall: 59.91 %
f1: 0.655660377358
AUC: 0.79605391047
Confusion matrix:
[[7486   53]
 [  93  139]]
MCC: 0.649150048349


Finally, the log regression:

In [26]:
evaluation(Xtest,Ytest,Ypredreg)

accuracy: 97.93 %
precision: 73.51 %
recall: 47.84 %
f1: 0.579634464752
AUC: 0.736571266197
Confusion matrix:
[[7499   40]
 [ 121  111]]
MCC: 0.583346900674


You should not be confused with a value of accuracy which is so high just due to the unbalanced dataset. Precision, recall and f1 score (as well as other metrics) show similar performance for all the models (a bit lower for log regression). The MCC value shows that our predictions are not random.

Remark: though variables X3 and Y3 are correlated (with coefficient approx 0.82) we do not to combine them into a single model as usually some additional knowledge of variables domain is needed to do so. Without explicit knowledge of the dependencies it is prefered to treat outputs separately.